In [3]:
from layers.dynamic_rnn import DynamicLSTM
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class DependencyProximity(nn.Module):
    def __init__(self, opt):
        self.opt = opt
        super(DependencyProximity, self).__init__()

    def forward(self, x, aspect_double_idx, text_len, aspect_len, dependency_dist):
        batch_size, seq_len = x.shape[0], x.shape[1]
        # print(type(aspect_double_idx), type(text_len), type(aspect_len), type(dependency_dist), type(batch_size), type(seq_len))
        weight = self.weight_matrix(aspect_double_idx, text_len, aspect_len, dependency_dist, batch_size, seq_len).to(torch.device('cpu'))
        x = weight.unsqueeze(2)*x
        return x

    def weight_matrix(self, aspect_double_idx, text_len, aspect_len, dependency_dist, batch_size, seq_len):
        aspect_double_idx = aspect_double_idx.cpu().numpy()
        text_len = text_len.cpu().numpy()
        aspect_len = aspect_len.cpu().numpy()
        dependency_dist = dependency_dist.cpu().numpy()
        weight = [[] for i in range(batch_size)]
        for i in range(batch_size):
            context_len = text_len[i] - aspect_len[i]
            for j in range(aspect_double_idx[i,0]):
                weight[i].append(1-dependency_dist[i,j]/context_len)
            for j in range(aspect_double_idx[i,0], aspect_double_idx[i,1]+1):
                weight[i].append(0)
            for j in range(aspect_double_idx[i,1]+1, text_len[i]):
                weight[i].append(1-dependency_dist[i,j]/context_len)
            for j in range(text_len[i], seq_len):
                weight[i].append(0)
        return torch.tensor(weight)

class PWCN_DEP_modified(nn.Module):
    def __init__(self, embedding_matrix, opt):
        super(PWCN_DEP_modified, self).__init__()
        self.opt = opt
        # self.embed = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float))
        # self.embed_dropout = nn.Dropout(0.3)
        self.txt_lstm = DynamicLSTM(768, 768, num_layers=1, batch_first=True, bidirectional=True)
        self.proximity = DependencyProximity(opt)
        self.convs3 = nn.Conv1d(2*768, 2*768, 3, padding=1).double()
        self.fc = nn.Linear(2*768, 3).double()

    def forward(self, inputs):
        text_indices, aspect_indices, left_indices, dependency_dist, bert_embedding = inputs
        # print(bert_embedding.shape)

        txt_len = torch.sum(text_indices != 0, dim=1)
        asp_len = torch.sum(aspect_indices != 0, dim=1)
        left_len = torch.sum(left_indices != 0, dim=-1)
        aspect_double_idx = torch.cat([left_len.unsqueeze(-1), (left_len+asp_len-1).unsqueeze(-1)], dim=-1)
        txt_out = bert_embedding
        # print(txt_out.shape, txt_len.shape)
        txt_out, (_, _) = self.txt_lstm(txt_out, txt_len)
        z = F.relu(self.convs3(
            self.proximity(txt_out, aspect_double_idx, txt_len, asp_len, dependency_dist).transpose(1, 2)))  # [(N,Co,L), ...]*len(Ks)
        z = F.max_pool1d(z, z.size(2)).squeeze(2)
        out = self.fc(z)
        return out

In [7]:
# -*- coding: utf-8 -*-


import os
import pickle
import torch
import torch.nn.functional as F
import argparse

from data_utils import ABSADatesetReader, ABSADataset, Tokenizer, build_embedding_matrix
from bucket_iterator import BucketIterator
from models import LSTM, PWCN_POS, PWCN_DEP
from dependency_dist import dependency_dist_func
from transformers import BertTokenizer, BertModel

class Inferer:
    """A simple inference example"""
    def __init__(self, opt):
        self.opt = opt
        print("loading {0} tokenizer...".format(opt.dataset))
        with open(opt.dataset+'_word2idx.pkl', 'rb') as f:
            word2idx = pickle.load(f)
            self.tokenizer = Tokenizer(word2idx=word2idx)
        embedding_matrix = build_embedding_matrix(self.tokenizer.word2idx, opt.embed_dim, opt.dataset)
        self.model = opt.model_class(embedding_matrix, opt)
        print('loading model {0} ...'.format(opt.model_name))
        self.model.load_state_dict(torch.load(opt.state_dict_path, map_location=lambda storage, loc: storage))
        # switch model to evaluation mode
        self.model.eval()
        torch.autograd.set_grad_enabled(False)

    def evaluate(self, raw_text, aspect):
        text_seqs = [self.tokenizer.text_to_sequence(raw_text.lower().strip())]
        aspect_seqs = [self.tokenizer.text_to_sequence(aspect.lower())]
        left_seqs = [self.tokenizer.text_to_sequence(raw_text.lower().split(aspect.lower())[0])]
        text_indices = torch.tensor(text_seqs, dtype=torch.int64)
        aspect_indices = torch.tensor(aspect_seqs, dtype=torch.int64)
        left_indices = torch.tensor(left_seqs, dtype=torch.int64)
        dependency_dist = torch.tensor([dependency_dist_func(raw_text, aspect)], dtype=torch.int64)

        tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
        bert_model = BertModel.from_pretrained("bert-base-uncased")
            
        inputs = tokenizer_bert.encode_plus(full_text, return_tensors="pt")
        outputs = bert_model(**inputs)

        last_hidden_states = outputs[0]
        
        data = {
            'text_indices':text_indices, 
            'aspect_indices':aspect_indices, 
            'left_indices':left_indices,
            'dependency_dist':dependency_dist,
            'bert_embedding': last_hidden_states,
        }
        t_inputs = [data[col] for col in self.opt.inputs_cols]
        t_outputs = self.model(t_inputs)

        t_probs = F.softmax(t_outputs, dim=-1).cpu().numpy()
        return t_probs

if __name__ == '__main__':
    model_classes = {
        'lstm': LSTM,
        'pwcn_pos': PWCN_POS,
        'pwcn_dep': PWCN_DEP,
        'pwcn_dep_BERT': PWCN_DEP_modified,
    }
    dataset = 'restaurant'
    # set your trained models here
    model_state_dict_paths = {
        'lstm': 'state_dict/lstm_'+dataset+'.pkl',
        'pwcn_pos': 'state_dict/pwcn_pos_'+dataset+'.pkl',
        'pwcn_dep': 'state_dict/pwcn_dep_'+dataset+'.pkl',
        'pwcn_dep_BERT': 'state_dict\pwcn_dep_restaurant_BERT_FINAL.pkl'
    }
    input_colses = {
        'lstm': ['text_indices'],
        'pwcn_pos': ['text_indices', 'aspect_indices', 'left_indices'], 
        'pwcn_dep': ['text_indices', 'aspect_indices', 'left_indices', 'dependency_dist'],
        'pwcn_dep_BERT': ['text_indices', 'aspect_indices', 'left_indices', 'dependency_dist', 'bert_embedding'],
    }
    class Option(object): pass
    opt = Option()
    opt.model_name = 'pwcn_dep_BERT'
    opt.model_class = model_classes[opt.model_name]
    opt.inputs_cols = input_colses[opt.model_name]
    opt.dataset = dataset
    opt.state_dict_path = model_state_dict_paths[opt.model_name]
    opt.embed_dim = 768
    opt.hidden_dim = 768
    opt.polarities_dim = 3
    opt.device = torch.device('cpu')

    inf = Inferer(opt)
    t_probs = inf.evaluate("great food but the service was dreadful!", 'service')    

    print(t_probs)
    print(t_probs.argmax(axis=-1))





loading restaurant tokenizer...
loading embedding_matrix: 768_restaurant_embedding_matrix.pkl
loading model pwcn_dep_BERT ...


RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED at ..\caffe2\serialize\inline_container.cc:132, please report a bug to PyTorch. Attempted to read a PyTorch file with version 3, but the maximum supported version for reading is 2. Your PyTorch installation may be too old. (init at ..\caffe2\serialize\inline_container.cc:132)
(no backtrace available)